# Imports

In [1]:
import pandas as pd
pd.options.display.max_columns = 50

import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import calendar
%matplotlib inline

data = pd.read_csv('../data/investments_VC.csv', encoding='iso-8859-1')
data.dropna(axis=0, how='all', inplace=True)

import warnings
warnings.simplefilter("ignore")

# Data Cleaning

In [2]:
data.columns = data.columns.str.replace(' ', '') 

## Converting to proper datatypes

In [3]:
data[["founded_at"]] = data[["founded_at"]].apply(pd.to_datetime, errors='coerce')
data[["last_funding_at"]] = data[["last_funding_at"]].apply(pd.to_datetime, errors='coerce')
data[["founded_month"]] = data[["founded_month"]].apply(pd.to_datetime, errors='coerce')
data[["first_funding_at"]] = data[["first_funding_at"]].apply(pd.to_datetime, errors='coerce')

In [4]:
data['funding_total_usd'] = data['funding_total_usd'].str.replace(',', '')

## Dropping Observations with Null Values

In [5]:
data = data.dropna(subset=['founded_year',
                           'first_funding_at',
                           'founded_quarter',
                           'founded_month',
                           'founded_at',
                           'state_code',
                           'country_code',
                           'status',
                           'market'
                   ])

## Removing quarter from founding quarter

In [6]:
data['founded_quarter'] = data.founded_quarter.str.split('-').str[1]

## Removing null values from founded year

All observations have a founded_at value, but not all values have a founded_year value. Therefore, we will use founded_at to fill null values in founded_year

In [7]:
data['founded_year'] = pd.DatetimeIndex(data['founded_at']).year

## Turning Category List into Individual Columns

Turn each value into a list

In [8]:
data.category_list = data.category_list.astype(str)

Split the list on |

In [9]:
data.category_list = data.category_list.apply(lambda x: x.split('|'))

Reset index 

In [10]:
data.reset_index(inplace = True, drop = True)

Strip whitespace from value lists

In [11]:
for i, item in enumerate(data.category_list):
    data.category_list[i] = [x.strip() for x in data.category_list[i] if x.strip()]

Making category list lowercase and splitting on spaces

In [12]:
for num in range(0, len(data.category_list)):
    data.category_list[num] = " ".join(data.category_list[num]).split()
    for i,elem in enumerate(data.category_list[num]):
        data.category_list[num][i] = data.category_list[num][i].lower()

Creating new dummy columns for category list

In [13]:
s = data.category_list

In [14]:
category_dummies = pd.get_dummies(s.apply(pd.Series).stack()).sum(level=0)

Concatonating dummy list with dataset

In [15]:
data = pd.concat([data,category_dummies], axis=1)

In [16]:
data.shape

(23238, 772)

# Feature Engineering

## Adding Domain name ending as a feature

In [17]:
#split on the domain after last .
end = data["homepage_url"].str.rsplit(".", n = 1, expand = True)
data['url_ending'] = end[1]
# data.url_ending = data['url_ending'].str.rstrip('/')

#remove backslash values in the endings
slashed_vals = data['url_ending'].str.rsplit("/", n = 3, expand = True)
data['url_ending'] = slashed_vals[0]

## New Feature: days_from_founding_to_funding

Difference btw founding and funding date

In [18]:
data['days_from_founding_to_funding'] = data['first_funding_at']-data['founded_at']

Changing datatype to int

In [19]:
data['days_from_founding_to_funding'] = data['days_from_founding_to_funding'].dt.days.astype('int16')
data.funding_rounds = data.funding_rounds.astype(int)

In [20]:
data.head(2)

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,undisclosed,convertible_note,debt_financing,angel,...,visual,visualization,voip,waste,water,wealth,wear,web,webos,weddings,wellness,wholesale,wind,windows,wine,wireless,women,workforces,world,worlds,worldwide,writers,young,url_ending,days_from_founding_to_funding
0,/organization/waywire,#waywire,http://www.waywire.com,"[entertainment, politics, social, media, news]",News,1750000,acquired,USA,NY,New York City,New York,1,2012-06-01,2012-06-01,Q2,2012,2012-06-30,2012-06-30,1750000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,com,29
1,/organization/r-ranch-and-mine,-R- Ranch and Mine,NaN,"[tourism, entertainment, games]",Tourism,60000,operating,USA,TX,Dallas,Fort Worth,2,2014-01-01,2014-01-01,Q1,2014,2014-08-17,2014-09-26,0.0,0.0,60000.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,228


## New Feature: time_between_first_and_last_funding

Time between first and last funding round

In [21]:
data['time_between_first_and_last_funding'] = data['last_funding_at']-data['first_funding_at']
data['time_between_first_and_last_funding'] = data['time_between_first_and_last_funding'].dt.days.astype('int16')
data.time_between_first_and_last_funding = data.time_between_first_and_last_funding.astype(int)

## Dummies

We are going to dummy the following categorical columns: market, country_code, state_code, funding_rounds, founded_year, url_ending 

### Founding Month

Turn month integer into abbreviation of month name

In [22]:
data['founding_month'] = pd.DatetimeIndex(data['founded_month']).month
data['founding_month'] = data['founding_month'].fillna(0)
data.founding_month = data.founding_month.astype(int)
data['founding_month'] = data['founding_month'].apply(lambda x: calendar.month_abbr[x])

Create dummy columns for each month

In [23]:
month_dummies = pd.get_dummies(data.founding_month, prefix='month')
data = data.drop(columns = ['founded_month','founding_month'])
data = pd.concat([data,month_dummies], axis=1)
data = data.drop(columns = ['month_Jan'])

### Founded Quarter

In [24]:
founded_quarter_dummies = pd.get_dummies(data.founded_quarter, drop_first=True, prefix='founded_quarter')
data = data.drop(columns = 'founded_quarter')
data = pd.concat([data,founded_quarter_dummies], axis=1)

### Country Code

In [25]:
country_code_dummies = pd.get_dummies(data.country_code, drop_first=True, prefix='country_code')
data = data.drop(columns = 'country_code')
data = pd.concat([data,country_code_dummies], axis=1)

### State Code

In [26]:
state_code_dummies = pd.get_dummies(data.state_code, drop_first=True, prefix='state_code')
data = data.drop(columns = 'state_code')
data = pd.concat([data,state_code_dummies], axis=1)

### Founded Year

In [27]:
founded_year_dummies = pd.get_dummies(data.founded_year, drop_first=True, prefix='founded_year')
data = data.drop(columns = 'founded_year')
data = pd.concat([data,founded_year_dummies], axis=1)

### Url Ending

In [28]:
url_dict = data.url_ending.value_counts().to_dict()

In [29]:
names_list = []

for key,value in url_dict.items():
    
    if value >= 5:
        names_list.append(key)
    else:
        pass

In [30]:
data.url_ending = data.url_ending.apply(lambda x: x if x in names_list else 'other')

In [31]:
url_ending_dummies = pd.get_dummies(data.url_ending, drop_first=True, prefix='url_ending')
data = data.drop(columns = 'url_ending')
data = pd.concat([data,url_ending_dummies], axis=1)

# Dropping Columns

In [32]:
data = data.drop(columns=['homepage_url',
                          'permalink',
                          'name',
                          'market',
                          'region',
                          'city',
                          'post_ipo_equity',
                          'post_ipo_debt',
                          'secondary_market',
                          'product_crowdfunding',
                          'round_C',
                          'round_D',
                          'round_E',
                          'round_F',
                          'round_G',
                          'round_H',
                          'category_list',
                          '&',
                          '+',
                          '7',
                          'of',
                          'on',
                          'non',
                          'qr',
                          'q&a',
                          'to',
                          'things',
                          'money',
                          'moneymaking',
                          'founded_at', 
                          'last_funding_at',
                          'funding_rounds',
                          'first_funding_at'
                          
])

In [33]:
data.columns.to_list()

['funding_total_usd',
 'status',
 'seed',
 'venture',
 'equity_crowdfunding',
 'undisclosed',
 'convertible_note',
 'debt_financing',
 'angel',
 'grant',
 'private_equity',
 'round_A',
 'round_B',
 '2.0',
 '3d',
 'accessories',
 'accounting',
 'active',
 'ad',
 'adherence',
 'adults',
 'advanced',
 'adventure',
 'advertising',
 'advice',
 'aerospace',
 'agent',
 'agriculture',
 'algorithms',
 'all',
 'alumni',
 'analyt',
 'analytics',
 'and',
 'android',
 'angels',
 'animal',
 'apis',
 'app',
 'application',
 'applications',
 'apps',
 'aquaculture',
 'architecture',
 'archiving',
 'art',
 'artificial',
 'artists',
 'assessment',
 'asset',
 'assisitive',
 'auctions',
 'audio',
 'augmented',
 'auto',
 'automated',
 'automation',
 'automotive',
 'b2b',
 'babies',
 'baby',
 'bananas',
 'banking',
 'based',
 'batteries',
 'beauty',
 'beer',
 'behavior',
 'benefits',
 'bicycles',
 'big',
 'billing',
 'bio-pharm',
 'bioinformatics',
 'biology',
 'biometrics',
 'biotechnology',
 'bitcoin',
 'b

# DataFrame to CSV

In [34]:
data = data.loc[data['funding_total_usd'] != ' -   ']

In [35]:
status = []

for val in data['status']:
    if val == 'acquired':
        status.append(1)
    elif val == 'operating':
        status.append(0)
    else:
        status.append(0)
data['target'] = status        

data.drop(['status'], axis=1, inplace=True)

In [36]:
data.funding_total_usd = data['funding_total_usd'].astype(int)

In [38]:
data.to_csv('final_startup_data.csv')

In [39]:
data.target.value_counts()

0    17845
1     2125
Name: target, dtype: int64